In [5]:
! wget http://tc11.cvc.uab.es/index.php?com=upload&action=file_down&section=dataset&section_id=106&file=112

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.feature_extraction.text import CountVectorizer
# https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
"""the encoder is a stack of bidirectional
GRUs while the decoder combines a unidirectional GRU and
an attention mechanism into the recurrent sequence genera-
tor.

we use
cross-entropy (CE) as the criterion
"""
criterion = nn.CrossEntropyLoss()
encoder = nn.GRU(,, 4, True, False, 0, True)
m, n = 256, 256
nPrime, D = 500, 500
optimizer = optim.Adadelta(params, lr=1.0, rho=0.95, eps=1e-06)

In [ ]:
class EquationNet(nn.Module):
    def __init__(self):
        super(EquationNet,self,embedding_dim,vocab_size).__init__()
        #encoder
        self.conv1=nn.Conv2d(1,64,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool1=nn.MaxPool2d((2,2),stride=(2,2))
        self.conv2=nn.Conv2d(64,128,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool2=self.pool1=nn.MaxPool2d((2,2),stride=(2,2))
        self.conv3=nn.Conv2d(128,256,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.bn3=nn.BatchNorm2d(256)
        self.conv4=nn.Conv2d(256,256,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool4=self.pool1=nn.MaxPool2d((2,1),stride=(2,1))
        self.conv5=nn.Conv2d(256,512,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.bn5=nn.BatchNorm2d(512)
        self.pool5=self.pool1=nn.MaxPool2d((2,1),stride=(2,1))
        self.conv6=nn.Conv2d(512,512,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.bn6=nn.BatchNorm2d(512)
        #embeddings
        self.embed=nn.linear(512,embedding_dim)
        #decoder
        self.blstm=nn.LSTM(input_size=embedding_dim,hidden_size=vocab_size,num_layers=2,batch_first=True,dropout=0.1,bidirectional=True)
    
    def forward(self,inputs):
        #encoding
        #tensor has size (batch_size, color_chans, height, width)
        x=self.conv1(inputs)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.pool2(x)
        x=self.conv3(x)
        x=self.bn3(x)
        x=self.conv4(x)
        x=self.pool4(x)
        x=self.conv5(x)
        x=self.bn5(x)
        x=self.pool5(x)
        x=self.conv6(x)
        x=self.bn6(x)
        #tensor has size (batch_size, features, new_height, new_width)
        #converting 2D to 1D
        x=torch.flatten(x,start_dim=2,end_dim=3)
        #tensor has size (batch_size, features, new_height*new_width)
        #transposing to work on features
        x=torch.transpose(x,1,2)
        #embedding
        #tensor has size (batch_size, new_height*new_width, features)
        x=self.embed(x)
        #decoding
        #tensor has size (batch_size, new_height*new_width, embedding_dim)
        x=self.blstm(x)
        #tensor has size (batch_size, new_height*new_width, 2*vocab_size)
        x1=x[:,:,:hidden_size]
        x2=x[:,:,hidden_size:2*hidden_size]
        x=x1+x2
        #tensor has size (batch_size, new_height*new_width, vocab_size)
        x=F.gumbel_softmax(x,hard=True,dim=3)
        return x

In [ ]:
def train_net(model, data_loader, loss, num_epochs, pad_value, verbose=True):
    losses=[]
    for epoch in range(num_epochs):
        for sample_id, (sample_data, sample_truth) in enumerate(data_loader):
            output=model(sample_data)
            out_len=output.shape[0]
            truth_len=sample_truth.shape[0]
            if out_len>truth_len:
                modified_truth=F.pad(sample_truth,(0,out_len-truth_len))
                for i in range(out_len-truth_len):
                    modified_truth[truth_len+i]=pad_value
            elif out_len<truth_len:
                raise CapacityError
            else:
                modified_truth=sample_truth
            modified_truth=adapt_truth(sample_truth, output)
            optimizer.zero_grad()
            loss_value=loss(output, modified_truth)
            losses.append(loss_value)
            loss.backward()
            optimizer.step()
            if verbose and ((i+1)%len(data_loader))==0:
                print("Epoch[{}/{}], batch[{}/{}], loss : {}".format(epoch+1,num_epochs, batch_id, len(data_loader), loss_value))
    return losses